In [ ]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [8]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.5, server:0.30.5

### Test Calls

#### Legacy, but works

In [40]:
r = localAPI.run(test01
               ,runAssump = [("call", {"or":[{"afterDate":"2023-06-01"} # clean up any of them met
                                             ,{"poolFactor":0.3}]}
                             )]
               ,read=True)

r['result']['status']

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2023-04-20,Amortizing,Called
2,2023-04-20,DealEnd,Clean Up


### New `callWhen`


`callWhen` has two options:

* `OnDates` which the `condition` list will be tested on the dates described by a `DatePattern`
* `if` which the `condition` list will be tested on the waterfall payment dates.

#### Call On Dates


In [37]:
r = localAPI.run(test01
               ,runAssump = [("callWhen", ("onDates", "MonthEnd", ["all", [("poolFactor",),"<",0.5]
                                                                        , ["date",">","2022-01-01"]
                                                                  ])
                             )]
               ,read=True)
r['result']['status']

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2022-11-30,Amortizing,Called
2,2022-11-30,DealEnd,Clean Up


#### Call by `Condition`

In [41]:
r = localAPI.run(test01
               ,runAssump = [("callWhen", ("if", ["all", [("poolFactor",),"<",0.2]
                                                         , ["date",">","2022-01-01"]
                                                  ])
                             )]
               ,read=True)
r['result']['status']

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2023-06-20,Amortizing,Called
2,2023-06-20,DealEnd,Clean Up


#### Combine of both

In [42]:
r = localAPI.run(test01
               ,runAssump = [("callWhen", ("if", ["all", [("poolFactor",),"<",0.2]
                                                         , ["date",">","2022-01-01"]
                                                  ])
                                         , ("if", ["all", [("poolFactor",),"<",0.3]
                                                         , ["date",">","2022-01-01"]
                                                  ])                              
                             )]
               ,read=True)
r['result']['status']

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2023-04-20,Amortizing,Called
2,2023-04-20,DealEnd,Clean Up


In [43]:
r = localAPI.run(test01
               ,runAssump = [("callWhen", ("if", ["all", [("poolFactor",),"<",0.2]
                                                         , ["date",">","2022-01-01"]
                                                  ])
                                         ,("onDates", "MonthEnd", ["all", [("poolFactor",),"<",0.5]
                                                                        , ["date",">","2022-01-01"]
                                                                  ])                             
                             )]
               ,read=True)
r['result']['status']

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed

,Date,From,To
0,2021-06-15,PreClosing,Amortizing
1,2022-11-30,Amortizing,Called
2,2022-11-30,DealEnd,Clean Up
